In [1]:
import os, re, os.path, json, sys
import argparse
from tqdm import tqdm
from glob import glob
from pydub import silence, AudioSegment
from pytube import YouTube

In [9]:
# in_dir = sys.argv[1]
# out_dir = sys.argv[2]
in_dir = 'C:/Users/dataedu09/Documents/이동진/workplace/code/program_audio'
out_dir ='C:/Users/dataedu09/Documents/이동진/workplace/code/002'

big_dir = glob(in_dir+'/*.mp4')
big_dir.sort()
for nu, big in enumerate(big_dir[:5]):
    num = str(nu + 306)
    b_name = os.path.basename(big)
    name = b_name.split('.')[0]
    trans_big = '%s/%s.wav'%(in_dir, name)
    os.system('ffmpeg -i '+big+' -acodec pcm_s16le -ac 1 -ar 24000 '+trans_big)
    audio_file = AudioSegment.from_file(trans_big)
    skip_idx = 0
    min_silence_len = 450
    silence_thresh=-40
    silence_chunk_len=100
    keep_silence=100
    out_ext = 'wav'
    not_silence_ranges = silence.detect_nonsilent(audio_file,min_silence_len=silence_chunk_len, silence_thresh=silence_thresh)
    edges = [not_silence_ranges[0]]
    for idx in range(1, len(not_silence_ranges)-1):
        cur_start = not_silence_ranges[idx][0]
        prev_end = edges[-1][1]

        if cur_start - prev_end < min_silence_len:
            edges[-1][1] = not_silence_ranges[idx][1]
        else:
            edges.append(not_silence_ranges[idx])

    audio_paths = []
    os.makedirs('%s/%s'%(out_dir,num), exist_ok=True)
    for idx, (start_idx, end_idx) in enumerate(edges[skip_idx:]):
        export_dir = "{}/{}/{}_{:04d}.wav".format(out_dir, num, num, idx)
        start_idx = max(0, start_idx - keep_silence)
        end_idx += keep_silence

        target_audio_path = export_dir
        #target_audio_path = "{}/{}.{:04d}.{}".format(
        #        out_dir, filename, idx, out_ext)

        segment=audio_file[start_idx:end_idx]

        segment.export(target_audio_path, out_ext)  # for soundsegment

        audio_paths.append(target_audio_path)


In [13]:
in_dir = 'C:/Users/dataedu09/Documents/이동진/workplace/code/program_audio'
big_dir = glob(in_dir+'/*.mp4')
big_dir.sort()
for nu, big in enumerate(big_dir[:2]):
    audio_file = AudioSegment.from_file(big)
    detect = silence.detect_silence(audio_file, min_silence_len=1000, silence_thresh=-40, seek_step=1)
    total = 0
    for i in detect:
        d = i[1]-i[0]
        total += d
    print(total/len(detect))
#     print(detect)
#     print('---------------')

1561.054054054054
1941.2179487179487


## 텍스트랑 영상 uid로 다운

In [14]:
def tubedown(uid_lists , export_dir):
    import os, os.path
    from glob import glob
    from pytube import YouTube
    
    tmp = uid_lists
    with open(tmp,'r',encoding='utf-8') as f:
        uid_list = f.readlines()
        export = export_dir
        os.makedirs(export, exist_ok=True)
        for uid in uid_list:
            ## text_vtt형식:
            os.system('youtube-dl --write-sub --sub-lang ko --sub-format srv3 -o '+export+' --skip-download https://www.youtube.com/watch?v='+uid)
            
            ## 영상 다운:
            yt = YouTube('https://www.youtube.com/watch?v='+uid)
            stream = yt.streams.all()[0] 
            filename= '%s'%uid
            stream.download(export, filename=filename)
            mp4_dir = export+'/'+filename+'.mp4'
            wav_dir=export+'/'+filename+'.wav'
            flac_dir= export+'/flac/'+filename+'.flac'
            os.system("ffmpeg -f s16le -ar 16000 -ac 1 -i "+mp4_dir+" "+wav_dir)
            os.system("ffmpeg -i " + wav_dir + " -c:a flac "+flac_dir)
            os.remove(mp4_dir)

## vtt 및 영상 폴더 분류

In [2]:
def classfic_folder(indir, outdir):
    import os, os.path
    from glob import glob
    import webvtt
    from pydub import silence, AudioSegment
    import re
    
    indir = indir
    outdir = outdir
    files = glob(indir+'/*.vtt')
    for nu, fi in enumerate(files):
        num = str(nu + 306)
        f_name = os.path.basename(fi)
        name = f_name.split('.')[0]
        os.makedirs('%s/%s'%(outdir,num), exist_ok=True)
        for c in webvtt.read(fi):
            tm = str(c)
            texts = tm.split(' ',2)[2]
            text = re.sub(r'\\n', ' ', texts)
            with open(outdir+'/%s/%s.trans.txt'%(num,num),'a',encoding='utf-8') as f:
                f.write(text+'\n')
                
        ## 임시         
        mp4_dir = indir+'/'+name+'.mp4'
        wav_dir=indir+'/'+name+'.wav'
        os.system("ffmpeg -i "+mp4_dir+" -vn -acodec pcm_s16le -ar 44100 -ac 2 "+wav_dir)
#         os.remove(mp4_dir)
        
        audio_dir='%s/%s.wav'%(indir, name)        
        audio_file = AudioSegment.from_file(audio_dir)
        skip_idx = 0
        min_silence_len = 500
        silence_thresh=-35
#         silence_chunk_len=500
        keep_silence=100
        out_ext = 'wav'
        not_silence_ranges = silence.detect_nonsilent(audio_file,min_silence_len=min_silence_len, silence_thresh=silence_thresh)
        edges = [not_silence_ranges[0]]
        for idx in range(1, len(not_silence_ranges)-1):
            cur_start = not_silence_ranges[idx][0]
            prev_end = edges[-1][1]

            if cur_start - prev_end < min_silence_len:
                edges[-1][1] = not_silence_ranges[idx][1]
            else:
                edges.append(not_silence_ranges[idx])

        audio_paths = []
        for idx, (start_idx, end_idx) in enumerate(edges[skip_idx:]):
            export_dir = "{}/{}/{}_{:04d}.wav".format(outdir, num, num, idx)
            start_idx = max(0, start_idx - keep_silence)
            end_idx += keep_silence

            target_audio_path = export_dir
            segment=audio_file[start_idx:end_idx]
            segment.export(target_audio_path, out_ext)  # for soundsegment
            audio_paths.append(target_audio_path)

In [3]:
classfic_folder(indir='C:/Users/dataedu09/Documents/이동진/workplace/code/program_audio', outdir='C:/Users/dataedu09/Documents/이동진/workplace/code/002')

In [ ]:
def text_refine(indir, outdir):
    import glob
    indir = indir
    outdir =outdir
    file_dir = glob.glob(indir+'/*')
    for i in file_dir:
        file = glob(i+'/*.txt')
        b_name = os.path.basename(i)
        with open(file[0],'r',encoding='utf-8') as f:
            tmp = f.readlines()
            os.rename(file, indir+'/tmp.txt')
            os.remove(indir+'/tmp.txt')
            for i in tmp:
                tm= re.sub(r'([0-9]).([0-9])', r'\1 점 \2',i)
                tm = re.sub('''[.|?|,|:|'|"]''','',tm)
                p = re.compile(r"[0-9][가-힣]|[a-z][가-힣]").findall(tm) 
                for i in p:
                    tm = re.sub(i[0]+i[1],i[0]+' '+i[1],tm)
                with open(outdir+'/%s_tran.txt'%b_name,'a',encoding='utf-8') as f:
                    f.write(text)

In [ ]:
text_refine(indir='C:/Users/dataedu09/Documents/이동진/workplace/code/002', outdir='C:/Users/dataedu09/Documents/이동진/workplace/code/002')